In [1]:
println("Start!!!")
// spark 初始化，为后续sql的运行准备环境
import $ivy.`com.alibaba:fastjson:1.2.70`
import com.alibaba.fastjson.{JSON, JSONObject}

import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark.sql._
import org.apache.spark.{SparkConf, SparkContext}
var user = sys.env.get("JUPYTERHUB_USER").get
val conf = {
   new SparkConf()
   .setAppName(s"Online-Feature-For-Bid-${user}")
   .setMaster("yarn")
   .set("spark.yarn.queue", "root.zw03.hadoop-waimaiadrd.etl") // spark 的队列，可以根据自己的需要设置
   .set("spark.yarn.archive", "viewfs://hadoop-meituan/user/hadoop-hdp/notebook/spark-jars/spark_libs.zip")
   .set("spark.yarn.stagingDir", "viewfs://hadoop-meituan/ghnn03/sparkstaging")
   .set("spark.executor.extraLibraryPath", "/opt/meituan/nodemanager/lib/native/:/opt/meituan/hadoop/lib/native/")
   .set("spark.driver.extraLibraryPath", "/opt/meituan/nodemanager/lib/native/:/opt/meituan/hadoop/lib/native/")
   .set("spark.executorEnv.JAVA_HOME", "'/usr/local/java18'")
   .set("spark.yarn.appMasterEnv.JAVA_HOME", "'/usr/local/java18'")
   .set("spark.sql.hive.metastorePartitionPruning", "true")
   .set("spark.shuffle.service.enabled", "true")
   .set("spark.dynamicAllocation.enabled", "true")
   .set("spark.dynamicAllocation.minExecutors", "600") //spark的执行Executors
   .set("spark.dynamicAllocation.maxExecutors","1500")
   .set("spark.executor.memory","16G")
   .set("spark.driver.memory","7G")
   .set("spark.input.dir.recursive","true")
   .set("spark.sql.ignore.existed.function.enable","true")
   .set("spark.yarn.maxAppAttempts", "2")
   .set("spark.noResourceMaxInterval","86400s")
   .set("spark.scheduler.mode", "FAIR")
   .set("spark.executor.cores","8")
   .set("spark.yarn.executor.memoryOverhead","32024")
   .set("spark.yarn.driver.memoryOverhead","32024")
   .set("spark.hadoop.hive.mt.renew.token.enable", "true")
}
val spark = {
 NotebookSparkSession
   .builder()
   .config(conf)
   .enableHiveSupport.getOrCreate
}
// 引入相关的库
import java.text.SimpleDateFormat
import java.util.Calendar
import ammonite.ops.ImplicitWd._
import ammonite.ops._
import sys.process._
import java.io._
spark.sparkContext.applicationId
var logo="AFO_RL"
// 引入相关的库
import java.text.SimpleDateFormat
import java.util.Calendar
import ammonite.ops.ImplicitWd._
import ammonite.ops._
import sys.process._
import java.io._
import java.text.SimpleDateFormat
import java.util.Date

// 通用udf注册
import scala.collection.mutable.ArrayBuffer
import scala.math
import scala.util.Random;
import java.lang.Math
import com.alibaba.fastjson.{JSON, JSONObject}

import org.apache.spark.sql.{DataFrame, Row}
import org.apache.spark.sql.types.{ArrayType, FloatType, DoubleType, IntegerType, LongType, StringType, StructField, StructType}

import scala.collection.immutable.Map
import scala.collection.mutable

def JSONObj2Map(jsonObj: JSONObject): Map[String, String] = {
    var map = mutable.Map[String, String]()
    if(!jsonObj.isEmpty){
        val itr: java.util.Iterator[String] = jsonObj.keySet().iterator()
        while (itr.hasNext) {
            val key = itr.next()
            map += ((key, jsonObj.getString(key)))
        }
    }
    map.toMap
}

import org.apache.hadoop.hive.ql.exec.UDF
import spark.implicits._
val findIndex2 =  (strQuote:String) => {
    val poi = strQuote.split("\073").last
    val ind = strQuote.split("\073").zipWithIndex.filter(_._1 == poi).map(_._2).min
    ind.toString
}
spark.sqlContext.udf.register("findIndex2", findIndex2)
spark.udf.register("findIndex2", findIndex2)

def getDate(delta:Int, current_day:String = "today"):String = {
    val sdf:SimpleDateFormat = new SimpleDateFormat("yyyyMMdd")
    val cal:Calendar = Calendar.getInstance()
    if ( !current_day.equals("today")){
      val dt = sdf.parse(current_day)
      cal.setTime(dt)
    }
    cal.add(Calendar.DATE,delta)
    val day = sdf.format(cal.getTime)
    day
}
def getTimeSuffix():String={
    val dateFormat = new SimpleDateFormat("yyyyMMdd")
    val cla = Calendar.getInstance()
    cla.setTimeInMillis(System.currentTimeMillis())
    val table_suffix = dateFormat.format(cla.getTime)
    table_suffix
}
def dropTable(sqlContext:SQLContext, tableName:String, is_execute_sql:Boolean):Unit={
    printCurTime()
    var sql = s"Drop table if exists ${tableName}"
    if(is_execute_sql){
        println(s"执行 \n ${sql}")
        sqlContext.sql(sql)
    }
    printCurTime()
}

def executeSQL(sqlContext:SQLContext, sql:String, is_execute_sql:Boolean):DataFrame={
    printCurTime()
    println(s"执行 \n ${sql}")
    var result_ = sqlContext.sql(sql)
    printCurTime()
    result_
}

def genDateList(start: String, end: String): List[String] = {
    import java.util.Date
    import org.joda.time.{DateTime, Instant, Days}
    import org.joda.time.format.{DateTimeFormat, DateTimeFormatter}

    val DATE_KEY_FORMAT: DateTimeFormatter = DateTimeFormat.forPattern("yyyyMMdd")
    val DATE_FORMAT: DateTimeFormatter = DateTimeFormat.forPattern("yyyyMMdd")

    val startDate = DATE_KEY_FORMAT.parseDateTime(start).toDateTime
    val endDate = DATE_KEY_FORMAT.parseDateTime(end).toDateTime

    val daysCount = Days.daysBetween(startDate, endDate).getDays() + 1
    (0 until daysCount).map(startDate.plusDays(_)).map(d => d.toString(DATE_FORMAT)).toList
  }



def printCurTime():Unit={
    println(new SimpleDateFormat("yyyy-MM-dd HH:mm:ss").format(new Date))
}


def showTopNFromSQL(sql:String, n:Int):Unit={
    printCurTime()
    var result = executeSQL(spark.sqlContext,sql,true)
    result.show(n)
    printCurTime()
}
def executeSQLLazy(sql:String):DataFrame={
    executeSQL(spark.sqlContext,sql,true)
}
def showTopNFromTableName(tableName:String, n:Int):Unit={
    printCurTime()
    var result = executeSQL(spark.sqlContext,s"""select * from ${tableName} limit ${n}""",true)
    result.show(n)
    printCurTime()
}
def executeShell(shell:String):Unit={
    printCurTime()
    println("----------------The content of shell is: -------------------")
    println(shell)
    println("----------------Start execution....-------------------")
    var file_name=System.currentTimeMillis()
    val writer = new PrintWriter(new File(s"/opt/meituan/dolphinfs_${user}/${file_name}.sh" ))
    writer.write(shell)
    writer.close()
    //启动脚本
    s"chmod +777 /opt/meituan/dolphinfs_${user}/${file_name}.sh".!
    s"sh /opt/meituan/dolphinfs_${user}/${file_name}.sh".!
    s"rm -r /opt/meituan/dolphinfs_${user}/${file_name}.sh".!
    println("----------------End-------------------")
    printCurTime()
}

def getFeatureMap(x:((String,String))):String={
    var x_1 = x._1
    var x_2 = x._2.replace("Type","")
    if (x_2.contains("Array")){
            var x_type = x_2.split('(')(1).split(",")(0)
            x_2  = s"""Array<${x_type}>"""
    }
    var x_3 = s"""${x_1} ${x_2}"""
    x_3
}
// 传入sql，返回结果表
def createTableFromSQLWithName(sql:String,tableName:String,database:String):Unit={
    var rowData=executeSQLLazy(sql)
    var tmpTableName=s"tmp_${System.currentTimeMillis()}"
    var finalTableName=database+"."+tableName
    var table_scheme = rowData.dtypes
    var columnsName = table_scheme.map{x=> x._1}.mkString(",")
    
    
    //var featureForCreateTable=table_scheme.map{x=> s"""${x._1} ${x._2.stripSuffix("Type")}""".replace("Type","")}.mkString(",")
    var featureForCreateTable=table_scheme.map(getFeatureMap).mkString(",")
    executeSQLLazy(s"""drop table if exists ${finalTableName}""")
    executeSQLLazy(s"""create table if not exists ${finalTableName}(${featureForCreateTable}) STORED AS ORC""")
    rowData.createOrReplaceTempView(tmpTableName)
    executeSQLLazy(s"insert OVERWRITE table ${finalTableName} select ${columnsName} from ${tmpTableName}")
}

def createTableFromSQL(sql:String):String={
    var finalTableName=s"${logo}_${System.currentTimeMillis()}"
    createTableFromSQLWithName(sql,finalTableName,"tmp")
    "tmp."+finalTableName
}
def createTableFromSQL(sql:String,database:String):String={
    var finalTableName=s"${logo}_${System.currentTimeMillis()}"
    createTableFromSQLWithName(sql,finalTableName,database)
    database+"."+finalTableName
}

def generateTrajectory(tableName:String, groupColumns:Array[String], rankColumns:Array[String], joinColumns:Array[String],database:String):String={
    
    println(tableName.split("."))
    var finalTableName=database+"."+tableName.split("\\.")(1)+"_trajectory"
    
    var table_scheme = executeSQLLazy(s"""select * from ${tableName} limit 1""").dtypes
    var columnsName = table_scheme.map{x=> x._1}.mkString(",")
    var groupColumnString=groupColumns.mkString(",")
    var rankColumnString = rankColumns.mkString(",")
    var joinConditionString = joinColumns.map(x=> s"t1.${x}=t2.${x} ").mkString("\n and ")
    var selectT1columnsNameString = table_scheme.map{x=> s"t1.${x._1}"}.mkString(",")
    var selectT2columnsNameString = table_scheme.map{x=> s"t2.${x._1} as next_${x._1}"}.mkString(",")
    //var featureForCreateTable=table_scheme.map(getFeatureMap).mkString(",")
    var featureForCreateTable=table_scheme.map(getFeatureMap).mkString(",") + ",rn int,"+table_scheme.map(x=> {
        var x_1 = x._1
        var x_2 = x._2.replace("Type","")
        if (x_2.contains("Array")){
            var x_type = x_2.split('(')(1).split(",")(0)
            x_2  = s"""Array<${x_type}>"""
        }
        
        s"""next_${x_1} ${x_2}"""}
                                              ).mkString(",")+",next_rn int"
    var allColumnString=table_scheme.map{x=> s"${x._1}"}.mkString(",")+",rn,"+table_scheme.map{x=> s"next_${x._1}"}.mkString(",")+",next_rn"
    var sql=s"""
        select ${selectT1columnsNameString},t1.rn,${selectT2columnsNameString},t2.rn as next_rn
        from(
            select ${columnsName}, RANK() OVER(PARTITION BY ${groupColumnString} ORDER BY ${rankColumnString} asc) AS rn 
            from ${tableName}
        )t1
        left join(
            select ${columnsName}, RANK() OVER(PARTITION BY ${groupColumnString} ORDER BY ${rankColumnString} asc) AS rn 
            from ${tableName}
        )t2
        on t1.rn+1 = t2.rn
        and ${joinConditionString}
        """
    var trajectoryData=executeSQLLazy(sql)
    var tmpTableName=s"tmp_${System.currentTimeMillis()}"
    executeSQLLazy(s"""drop table if exists ${finalTableName}""")
    executeSQLLazy(s"""create table if not exists ${finalTableName}(${featureForCreateTable}) STORED AS ORC""")
    trajectoryData.createOrReplaceTempView(tmpTableName)
    spark.sql(s"insert OVERWRITE table ${finalTableName} select ${allColumnString} from ${tmpTableName}")
    finalTableName
}
def createPTTableFromTable(tableName:String,targetTableName:String,database:String):String={
    var finalTableName=targetTableName
    var rowData=executeSQLLazy(s"select * from ${tableName} limit 1")
    var tmpTableName=s"${database}.${logo}_${System.currentTimeMillis()}"
    var table_scheme = rowData.dtypes
    var columnsName = table_scheme.map{x=> x._1}.mkString(",")
    var featureForCreateTable=table_scheme.map(getFeatureMap).mkString(",")
    executeSQLLazy(s"""drop table if exists ${finalTableName}""")
    executeSQLLazy(s"""create table if not exists ${finalTableName}(${featureForCreateTable})  PARTITIONED BY (pt string) STORED AS ORC""")
    rowData.createOrReplaceTempView(tmpTableName)
    var dt = "today"
    var date_s=getDate(-1,dt)
    spark.sql(s"INSERT OVERWRITE TABLE ${finalTableName} PARTITION (pt='${date_s}') select ${columnsName} from ${tmpTableName}")
    finalTableName
}

def createPTTableFromTableWithSpace(tableName:String,database:String):String={
    var finalTableName=database+s".${logo}_${System.currentTimeMillis()}"
    var rowData=executeSQLLazy(s"select * from ${tableName}")
    var tmpTableName=s"tmp_${System.currentTimeMillis()}"
    var table_scheme = rowData.dtypes
    var columnsName = table_scheme.map{x=> x._1}.mkString(",")
    var featureForCreateTable=table_scheme.map(getFeatureMap).mkString(",")
    executeSQLLazy(s"""drop table if exists ${finalTableName}""")
    executeSQLLazy(s"""create table if not exists ${finalTableName}(${featureForCreateTable})  PARTITIONED BY (pt string) STORED AS ORC""")
    rowData.createOrReplaceTempView(tmpTableName)
    var dt = "today"
    var date_s=getDate(-1,dt)
    spark.sql(s"INSERT OVERWRITE TABLE ${finalTableName} PARTITION (pt=${date_s}) select ${columnsName} from ${tmpTableName}")
    finalTableName
}

Start!!!


SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#noProviders for further details.
SLF4J: Class path contains SLF4J bindings targeting slf4j-api versions prior to 1.8.
SLF4J: Ignoring binding found at [jar:file:/conda/envs/notebook/share/jupyter/kernels/scala211/pack/lib/slf4j-nop-1.7.26.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#ignoredBindings for an explanation.


Adding Hadoop conf dir /opt/meituan/hadoop/etc/hadoop to classpath
Adding Hive conf dir /opt/meituan/spark-2.2/conf to classpath
Creating SparkSession


log4j:WARN No appenders could be found for logger (io.netty.util.internal.logging.InternalLoggerFactory).
log4j:WARN Please initialize the log4j system properly.
log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.


Spark UI

import $ivy.$                            

import com.alibaba.fastjson.{JSON, JSONObject}


import org.apache.log4j.{Level, Logger}

import org.apache.spark.sql._

import org.apache.spark.{SparkConf, SparkContext}

user: String = "wanghao251"
conf: SparkConf = org.apache.spark.SparkConf@659ff75e
spark: SparkSession = org.apache.spark.sql.SparkSession@754f24b2
import java.text.SimpleDateFormat

import java.util.Calendar

import ammonite.ops.ImplicitWd._

import ammonite.ops._

import sys.process._

import java.io._

res0_16: String = "application_1669189271912_2392066"
logo: String = "AFO_RL"
import java.text.SimpleDateFormat

import java.util.Calendar

import ammonite.ops.ImplicitWd._

import ammonite.ops._

import sys.process._

import java.io._

import java.text.SimpleDateFormat

import java.util.Date

// 通用udf注册

import scala.collection.mutable.ArrayBuffer

import scala.math

import scala.util.Random;

import java.lang.Math

import com.alibaba.fastjson.{JSON, JSONObject}


import or

In [17]:
// 需要修改的地方：训练配置区域
// var train_table_name=finalTrainTable//训练表名
var train_table_name = "mart_waimaiad.AFO_RL_1670836641297"
// var trainDt=s"""pt=${befor1day}"""
// var predictDt=s"""pt=${befor1day}"""
// var trainDt=s"""pt=20220928"""
// var predictDt=s"""pt=20220928"""
// var trainDt=s"""pt=20220913"""
// var predictDt=s"""pt=20220913"""

// var predict_table_name=finalPredictTable //评估表名 理论上不一样，当前就直接用了，正式使用的时候请使用另外一个表
var predict_table_name="mart_waimaiad.AFO_RL_1670915491071"
var cur_state_dynamic_fea="rawbid,precisebid,is_expose,is_click,final_charge,is_order,revenue,unused,click,user_tag_pref_click_3day,hour,user_ord_num_weekdays,charge_before2week,order_num_before2week,impr_before1week,feacomp_cvr,click_before1week,lowerk,real_cpc,real_csm,revenue_before3week,cpc_before1week,charge_before3week,winningscore_comp12,impr,real_cst_bdgt,real_aBln,feacomp_ctr,real_expose,real_revenue,revenue_before2week,ratioctr,real_price,lowerbid,ratio_aimcpc,cpc_before4week,order_num_before3week,histctr,mt_butie_before2week,mt_butie_before3week,real_budget,order_num_before4week,cpc,order_num_before1week,revenue_before1week,mt_butie,cpc_before2week,cpc_before3week,aimcpc,mt_butie_before1week,real_csm_bln,click_before3week,charge_before1week,click_before4week,click_before2week,real_click,winningscore_qs,cityid,ratio_click,ratioctr_exp,upperk,charge,revenue_before4week,ctr,order_num,impr_before4week,impr_before2week,mt_butie_before4week,real_order,ratio_cpc,impr_before3week,charge_before4week,real_bln".toLowerCase() //RL 状态特征
var next_state_dynamic_fea="next_rawbid,precisebid,next_is_expose,next_is_click,next_final_charge,next_is_order,next_revenue,next_unused,next_click,next_user_tag_pref_click_3day,next_hour,next_user_ord_num_weekdays,next_charge_before2week,next_order_num_before2week,next_impr_before1week,next_feacomp_cvr,next_click_before1week,next_lowerk,next_real_cpc,next_real_csm,next_revenue_before3week,next_cpc_before1week,next_charge_before3week,next_winningscore_comp12,next_impr,next_real_cst_bdgt,next_real_aBln,next_feacomp_ctr,next_real_expose,next_real_revenue,next_revenue_before2week,next_ratioctr,next_real_price,next_lowerbid,next_ratio_aimcpc,next_cpc_before4week,next_order_num_before3week,next_histctr,next_mt_butie_before2week,next_mt_butie_before3week,next_real_budget,next_order_num_before4week,next_cpc,next_order_num_before1week,next_revenue_before1week,next_mt_butie,next_cpc_before2week,next_cpc_before3week,next_aimcpc,next_mt_butie_before1week,next_real_csm_bln,next_click_before3week,next_charge_before1week,next_click_before4week,next_click_before2week,next_real_click,next_winningscore_qs,next_cityid,next_ratio_click,next_ratioctr_exp,next_upperk,next_charge,next_revenue_before4week,next_ctr,next_order_num,next_impr_before4week,next_impr_before2week,next_mt_butie_before4week,next_real_order,next_ratio_cpc,next_impr_before3week,next_charge_before4week,next_real_bln".toLowerCase() //RL 下一个状态特征
var reward="reward"//RL reward 字段名字
var poi_id="poi_id"// pv 后续会去掉，修改为key
var pvid="pvid"// pv 后续会去掉，修改为key
var action="action"//RL action 字段名字
var is_terminal="is_terminal" //RL terminal 字段
var path_prefix=s"viewfs://hadoop-meituan/user/hadoop-hmart-waimaiad/${user}/${train_table_name}" //模型和tf recoder位置
// var cur_date=befor1day
// var cur_date="20220928"
// var cur_date="20220913"
// var start_date=befor1day //训练表的分区
// var end_date=befor1day //训练表的分区
var start_date="20221207" //训练表的分区
var end_date="20221207" //训练表的分区
var cur_date="20221207"
var trainDt=s"""pt=20221207"""
var predictDt=s"""pt=20221207"""

var quote="\""

var model_name="R-BCQ" //是否使用REM
var i_loss_weight="1" //使用BCQ时iloss的权重
var i_regularization_weight="0.2" 
var q_loss_weight="1" //使用BCQ时qloss的权重
var use_bcorle="True"


var parameter_servers=1 //ps的数量
var workers=3 //work的数量
var account=user //AFO框架启动消息通知对象
var queue="root.zw05_training_cluster.hadoop-waimaiadrd.cpu_job"
// 模型配置区
var batch_size="64" // batch size
// var batch_size="128"
var epoch="3"
var learning_rate="0.00001"
var deep_layers="128,64,64" //网络设置
var gamma="1.0"
var num_action="20" //action的个数
var update_interval="300"
var embed_dim="8" //离散特征的embedding维度
var bit="22"//离散特征的空间
var ext_is_predict_serving="0"
var use_bn="False"

var num_heads="20" //REM head 个数
var threshold="0.3" // BCQ阈值
var use_cate_feature="True"
var use_dense_hashtable="True" //是否使用dense_hashtable
// var cur_state_cate_fea="poi_id,rn" //RL 状态特征
// var next_state_cate_fea="next_poi_id,next_rn" //RL 下一个状态特征
var cur_state_cate_fea="poi_id" //RL 状态特征
var next_state_cate_fea="poi_id" //RL 下一个状态特征
var model_file="udm/tangbo17/rbcq.py" //用户自定义模型代码文件
var readFromHive="True"
var file_path="rl_easy_go_low" // 模型的文件目录
var mod="local" //mod=local 代表启动workbench本地调试，其他代表提交到服务器
var code_branch = "wanghao"

train_table_name: String = "mart_waimaiad.AFO_RL_1670487390331"
predict_table_name: String = "mart_waimaiad.AFO_RL_1670488258785"
cur_state_dynamic_fea: String = "rawbid,precisebid,is_expose,is_click,final_charge,is_order,revenue,unused,click,user_tag_pref_click_3day,hour,user_ord_num_weekdays,charge_before2week,order_num_before2week,impr_before1week,feacomp_cvr,click_before1week,lowerk,real_cpc,real_csm,revenue_before3week,cpc_before1week,charge_before3week,winningscore_comp12,impr,real_cst_bdgt,real_abln,feacomp_ctr,real_expose,real_revenue,revenue_before2week,ratioctr,real_price,lowerbid,ratio_aimcpc,cpc_before4week,order_num_before3week,histctr,mt_butie_before2week,mt_butie_before3week,real_budget,order_num_before4week,cpc,order_num_before1week,revenue_before1week,mt_butie,cpc_before2week,cpc_before3week,aimcpc,mt_butie_before1week,real_csm_bln,click_before3week,charge_before1week,click_before4week,click_before2week,real_click,winningscore_qs,cityid,ratio_click,ratioctr_exp,upperk,

In [ ]:
// 训练程序 传入模型的参数
// ##################################################################
// FAQ
// Q1. 报错 ModuleNotFoundError: No module named 'pytalos' : A:在jupyter客户端的命令行执行pip安装命令： pip install pytalos
// Q2. Exit code: 253 Exception message: Launch container failed Shell output: main : command provided 1 main : run as user is hadoop-hmart-waimaiad main : requested yarn user is hadoop-hmart-waimaiad Getting exit code file. A2: source /opt/meituan/hadoop-gpu/bin/hadoop_user_login.sh hadoop-hmart-waimaiad
// ##################################################################
var task="train" //任务类型 train,predict
var with_chief="true"
epoch="1"
// var model_name_list=List("R-BCQ","BCQ","REM","DQN")
var model_name_list="R-BCQ"
mod="local1" 
var train_cmd=s"""
    if [[ ! -d "/opt/meituan/dolphinfs_${user}/${train_table_name}/mtjupter" ]]; 
    then
        echo "文件夹不存在"
        mkdir /opt/meituan/dolphinfs_${user}/${train_table_name}
        cd /opt/meituan/dolphinfs_${user}/${train_table_name}
        git clone -b wanghao ssh://git@git.sankuai.com/~tangbo17/mtjupter.git
        git checkout ${code_branch}
    else
        echo "文件夹存在"
    fi

    cd /opt/meituan/dolphinfs_${user}/${train_table_name}/mtjupter
    git pull
    ls
    cd OS/wanghao/${file_path}
    echo "开始train model"
    sh run.sh ${train_table_name} ${predict_table_name} ${model_name} ${batch_size} ${epoch} ${learning_rate} ${deep_layers} ${gamma} ${num_action} ${update_interval} ${embed_dim} ${bit} ${ext_is_predict_serving} ${use_bn} ${i_loss_weight} ${i_regularization_weight} ${q_loss_weight} ${num_heads} ${threshold} ${path_prefix} ${cur_date} ${cur_date} ${cur_state_dynamic_fea} ${next_state_dynamic_fea} ${task} ${parameter_servers} ${workers} ${account} ${queue} ${with_chief} ${use_dense_hashtable} ${cur_state_cate_fea} ${next_state_cate_fea} ${use_cate_feature} ${readFromHive} ${file_path} ${mod} ${train_table_name}/${trainDt} ${use_bcorle}
    """
executeShell(train_cmd)


2022-12-09 15:14:24
----------------The content of shell is: -------------------

    if [[ ! -d "/opt/meituan/dolphinfs_wanghao251/mart_waimaiad.AFO_RL_1670487390331/mtjupter" ]]; 
    then
        echo "文件夹不存在"
        mkdir /opt/meituan/dolphinfs_wanghao251/mart_waimaiad.AFO_RL_1670487390331
        cd /opt/meituan/dolphinfs_wanghao251/mart_waimaiad.AFO_RL_1670487390331
        git clone -b wanghao ssh://git@git.sankuai.com/~tangbo17/mtjupter.git
        git checkout wanghao
    else
        echo "文件夹存在"
    fi

    cd /opt/meituan/dolphinfs_wanghao251/mart_waimaiad.AFO_RL_1670487390331/mtjupter
    git pull
    ls
    cd OS/wanghao/rl_easy_go_low
    echo "开始train model"
    sh run.sh mart_waimaiad.AFO_RL_1670487390331 mart_waimaiad.AFO_RL_1670488258785 R-BCQ 64 1 0.00001 128,64,64 1.0 20 300 8 22 0 False 1 0.2 1 20 0.3 viewfs://hadoop-meituan/user/hadoop-hmart-waimaiad/wanghao251/mart_waimaiad.AFO_RL_1670487390331 20221207 20221207 rawbid,precisebid,is_expose,is_click,final_charge

美团代码仓库(DevTools Code)                                 代码仓库组
[连接信息]:         gh-proxy02 => 82a4b79e-be40-420b-9299-13d8c59234cf
[路由信息]:                                 slave-mcode_shard_03-gh-01
                                   


Already up-to-date.
common
OS
personal
PT
readme.md
RecCpc
test.py
开始train model


+ set -u
+ set -o pipefail
+ table_name=mart_waimaiad.AFO_RL_1670487390331
+ predict_table_name=mart_waimaiad.AFO_RL_1670488258785
+ algorithm_name=R-BCQ
+ batch_size=64
+ epoch=1
+ learning_rate=0.00001
+ deep_layers=128,64,64
+ gamma=1.0
+ num_action=20
+ update_interval=300
+ embed_dim=8
+ bit=22
+ ext_is_predict_serving=0
+ use_bn=False
+ i_loss_weight=1
+ i_regularization_weight=0.2
+ q_loss_weight=1
+ num_heads=20
+ threshold=0.3
+ path_prefix=viewfs://hadoop-meituan/user/hadoop-hmart-waimaiad/wanghao251/mart_waimaiad.AFO_RL_1670487390331
+ start_date=20221207
+ end_date=20221207
+ cur_state_dynamic_fea=rawbid,precisebid,is_expose,is_click,final_charge,is_order,revenue,unused,click,user_tag_pref_click_3day,hour,user_ord_num_weekdays,charge_before2week,order_num_before2week,impr_before1week,feacomp_cvr,click_before1week,lowerk,real_cpc,real_csm,revenue_before3week,cpc_before1week,charge_before3week,winningscore_comp12,impr,real_cst_bdgt,real_abln,feacomp_ctr,real_expose,real_reven

use_bcq:True
use_rem:True


...

train_data:viewfs://hadoop-meituan/user/hadoop-hmart-waimaiad/wanghao251/mart_waimaiad.AFO_RL_1670487390331/train_tfrecoder/20221207


+ echo train_data:viewfs://hadoop-meituan/user/hadoop-hmart-waimaiad/wanghao251/mart_waimaiad.AFO_RL_1670487390331/train_tfrecoder/20221207
+ echo predict_data:viewfs://hadoop-meituan/user/hadoop-hmart-waimaiad/wanghao251/mart_waimaiad.AFO_RL_1670487390331/predict_tfrecoder/20221207


predict_data:viewfs://hadoop-meituan/user/hadoop-hmart-waimaiad/wanghao251/mart_waimaiad.AFO_RL_1670487390331/predict_tfrecoder/20221207
ext_is_predict_serving:1


+ echo ext_is_predict_serving:1
+ case ${mod} in
+ hope run --xml conf/dist.xml --usergroup hadoop-hmart-waimaiad -Dworker.memory=10240 -Dafo.engine.wait_for_job_finished=true -Dafo.afo-base.image.name=registryonline-hulk.sankuai.com/custom_prod/com.sankuai.data.hadoop.gpu/data-hadoop-hmart-hdp_afo-base-readhive:v1.1.16 -Dafo.docker.image.name=registryonline-hulk.sankuai.com/custom_prod/com.sankuai.data.hadoop.gpu/data_cog_hive_cuda11.0-8b7809e8 -Ddistribute.mode=ps '-Dworker.script=python rl_code/main.py' '-Dparameter.server.script=python rl_code/main.py' '-Dchief.script=python rl_code/main.py' -Dboard.log_dir=viewfs://hadoop-meituan/user/hadoop-hmart-waimaiad/wanghao251/mart_waimaiad.AFO_RL_1670487390331/model/20221207/mart_waimaiad.AFO_RL_1670487390331_R-BCQ -Dparameter.servers=1 -Dargs.ext_is_predict_serving=1 -Dworkers=3 -Dwith.chief=true -Dafo.xm.notice.receivers.account=wanghao251 -Dafo.app.name=mart_waimaiad.AFO_RL_1670487390331_R-BCQ -Dqueue=root.zw05_training_cluster.hadoop-w

2022-12-09 15:14:27 [INFO] Staged file or directory: /opt/meituan/dolphinfs_wanghao251/mart_waimaiad.AFO_RL_1670487390331/mtjupter/OS/wanghao/rl_easy_go_low/conf/dist.xml
2022-12-09 15:14:27 [INFO] Job info initialized at /workdir/sankuai/.hope/template_jobinfo/c0c37e24c4826cb481450bb580643240
2022-12-09 15:14:27 [INFO] Staged file or directory: /opt/meituan/dolphinfs_wanghao251/mart_waimaiad.AFO_RL_1670487390331/mtjupter/OS/wanghao/rl_easy_go_low
2022-12-09 15:14:28 [INFO] Hope config check success
2022-12-09 15:14:28 [INFO] File name: /workdir/sankuai/.hope/job-1e512b36_7791_11ed_a659_161ba065b71f/dist.tar.gz
2022-12-09 15:14:28 [INFO] File size: 86612B
2022-12-09 15:14:28 [INFO] Uploading...
2022-12-09 15:14:28 [INFO] Upload success
**********************************
 [INFO] code version save success 
**********************************
************************************
 job submit success, run_id: 801618 
************************************
download file: http://s3plus.vip.sanku

In [6]:
var dt = "today"
dt = getDate(-1,dt)
showTopNFromSQL("""
select * from 
(
    select t1.pvid, t4.user_id, t1.uuid, t1.poi_id, t1.request_time, t1.rawbid, t1.precisebid, t1.demandbiztype, t1.supplybiztype,
        coalesce(is_expose, 0) as is_expose, 
        coalesce(is_click, 0) as is_click, 
        coalesce(final_charge, 0) as final_charge, 
        if(t3.ad_request_id is not null and t3.uuid is not null and t3.poi_id is not null, 1, 0) as is_order,
        if(t3.mt_charge_fee is not null and t3.total is not null, t3.mt_charge_fee + t3.total, 0) as revenue,   
        rt_features, raw_features, modelname, business, params
    from 
    (
        select pvid,
            pvid2, 
            user_id,
            uuid,
            request_time,
            index_array(split(info, '\073'), 0) as poi_id,
            index_array(split(info, '\073'), 1) as rawbid,
            index_array(split(info, '\073'), 2) as precisebid,
            demandbiztype,
            supplybiztype
        from 
        (
            select upstreampvid as pvid,
                pvid as pvid2,
                userid as user_id,
                uuid,
                requesttime as request_time,
                demandbiztype,
                supplybiztype,
                cross_cols("\073", "#", poiid, rawbid, precisebid) as infos
            from log.adt_dsa_pv
            where dt = '${dt}'
                and poiid IS NOT NULL
                and poiid <> ''
                and expparams <> ''
                and rawbid is not null 
                and rawbid <> ''
                and precisebid is not null 
                and precisebid <> ''
                and phasetype = 2
                and responsestatus = 'SUCCESS'
                and instr(_mt_servername, 'staging') <= 0
                and demandbiztype in (2,3,4,6)
                and supplybiztype in (1,2,3,4)
        )t
        lateral view explode(split(t.infos, '#')) infos_table as info
    )t1
    join 
    (
        select requestid,
            get_json_object(dims, '$.user_id') as user_id,
            get_json_object(dims, '$.poi_id') as poi_id,
            get_json_object(dims, '$.uuid') as uuid,
            if (fvcs is null or fvcs = '', fvcs, pvlog_decode(fvcs)) as rt_features,
            fvs2 as raw_features,26,23,moses__icon__drag
            modelname,
            business,
            params
        from log.wml_modelsever_feature_for_dsa
        where dt = '${dt}'
            and _mt_client_env_ = 'prod'
            and business in ('dsa_os_unify_model', 'dsa_ecpc')
    )t4
    on t1.pvid2 = t4.requestid and t1.uuid = t4.uuid and t1.poi_id = t4.poi_id
    left outer join
    (
        select ad_request_id, uuid, poi_id, 
            if(sum(if(act = 3, 1, 0)) > 0, 1, 0) as is_expose,
            if(sum(if(act = 2, 1, 0)) > 0, 1, 0) as is_click,
            sum(if(act = 2 and is_charge = 1, final_charge, 0)) as final_charge,
            sum(if(act = 3, bid, 0)) * 100 as bid
        from mart_waimai_dw_ad.fact_flow_ad_entry_mv
        where dt = '${dt}'
            and act in (2,3)
            and poi_id is not null 
            and uuid is not null
            and ad_request_id is not null
            and ad_request_id <> ''
        group by ad_request_id, uuid, poi_id
    )t2 
    on t1.pvid = t2.ad_request_id and t1.uuid = t2.uuid and t1.poi_id = t2.poi_id
    left outer join 
    (
        select ad_request_id, uuid, poi_id, mt_charge_fee, total
        from mart_waimai_dw_ad.fact_flow_ad_sdk_entry_order_view
        where dt = '${dt}'
            and is_ad = 1
            and ad_request_id is not null 
            and ad_request_id <> '' 
    
    )t3 
    on t1.pvid = t3.ad_request_id and t1.uuid = t3.uuid and t1.poi_id = t3.poi_id
)t 
where (is_expose > 0) or (is_expose = 0 and rand() < 0.1)
""",10)

2022-12-09 10:41:56
2022-12-09 10:41:56
执行 
 
select * from 
(
    select t1.pvid, t4.user_id, t1.uuid, t1.poi_id, t1.request_time, t1.rawbid, t1.precisebid, t1.demandbiztype, t1.supplybiztype,
        coalesce(is_expose, 0) as is_expose, 
        coalesce(is_click, 0) as is_click, 
        coalesce(final_charge, 0) as final_charge, 
        if(t3.ad_request_id is not null and t3.uuid is not null and t3.poi_id is not null, 1, 0) as is_order,
        if(t3.mt_charge_fee is not null and t3.total is not null, t3.mt_charge_fee + t3.total, 0) as revenue,   
        rt_features, raw_features, modelname, business, params
    from 
    (
        select pvid,
            pvid2, 
            user_id,
            uuid,
            request_time,
            index_array(split(info, '\073'), 0) as poi_id,
            index_array(split(info, '\073'), 1) as rawbid,
            index_array(split(info, '\073'), 2) as precisebid,
            demandbiztype,
            supplybiztype
        from 
        

: 

In [25]:
var tempTable = createTableFromSQL(s"""
select * from mart_waimaiad_test.dsa_model_fp_os_rl_log """)

2022-12-12 11:14:36
执行 
 
select * from mart_waimaiad_test.dsa_model_fp_os_rl_log 
2022-12-12 11:14:36
2022-12-12 11:14:36
执行 
 drop table if exists tmp.AFO_RL_1670814876498
2022-12-12 11:14:36
2022-12-12 11:14:36
执行 
 create table if not exists tmp.AFO_RL_1670814876498(pvid String,user_id Long,uuid String,poi_id Long,request_time Long,rawbid Double,precisebid Double,demandbiztype Long,supplybiztype Long,is_expose Long,is_click Long,final_charge Double,is_order Long,revenue Double,rt_features String,raw_features String,modelname String,business String,params String,dt String) STORED AS ORC
2022-12-12 11:14:36
2022-12-12 11:14:36
执行 
 insert OVERWRITE table tmp.AFO_RL_1670814876498 select pvid,user_id,uuid,poi_id,request_time,rawbid,precisebid,demandbiztype,supplybiztype,is_expose,is_click,final_charge,is_order,revenue,rt_features,raw_features,modelname,business,params,dt from tmp_1670814876576


sql at cmd19.sc:131

2008 / 2000

Moved: 'viewfs://hadoop-meituan/user/hive/warehouse/tmp.db/afo_rl_1670814876498' to trash at: hdfs://dfsrouter/user/hadoop-hmart-waimaiad/.Trash/Current
2022-12-12 11:28:41


tempTable: String = "tmp.AFO_RL_1670814876498"

In [26]:
var finalTrainTable = createPTTableFromTableWithSpace(tempTable,"mart_waimaiad")

2022-12-12 11:28:58
执行 
 select * from tmp.AFO_RL_1670814876498
2022-12-12 11:28:58
2022-12-12 11:28:58
执行 
 drop table if exists mart_waimaiad.AFO_RL_1670815738712
2022-12-12 11:28:58
2022-12-12 11:28:58
执行 
 create table if not exists mart_waimaiad.AFO_RL_1670815738712(pvid String,user_id Long,uuid String,poi_id Long,request_time Long,rawbid Double,precisebid Double,demandbiztype Long,supplybiztype Long,is_expose Long,is_click Long,final_charge Double,is_order Long,revenue Double,rt_features String,raw_features String,modelname String,business String,params String,dt String)  PARTITIONED BY (pt string) STORED AS ORC
2022-12-12 11:28:58


sql at cmd19.sc:301

2000 / 2000

finalTrainTable: String = "mart_waimaiad.AFO_RL_1670815738712"

In [3]:
showTopNFromSQL("""
select count(*) from mart_waimaiad.AFO_RL_1670815738712
""",10)

2022-12-12 14:24:21
2022-12-12 14:24:21
执行 
 
select count(*) from mart_waimaiad.AFO_RL_1670815738712

2022-12-12 14:24:22


show at cmd0.sc:161

2000 / 2000

[Stage 1:====================================================>(1999 + 1) / 2000]

show at cmd0.sc:161

1 / 1

+----------+
|  count(1)|
+----------+
|1196937197|
+----------+

2022-12-12 14:25:22
